In [14]:
import sys
sys.path.append("../")
import pyradsmip
from pyradsmip import pyrads
import numpy as np
from scipy.integrate import trapz,simps,cumtrapz

In [25]:
class Dummy:
    pass

params = Dummy()

params.Rv = pyrads.phys.H2O.R # moist component
params.cpv = pyrads.phys.H2O.cp
params.Lvap = pyrads.phys.H2O.L_vaporization_TriplePoint
params.satvap_T0 = pyrads.phys.H2O.TriplePointT
params.satvap_e0 = pyrads.phys.H2O.TriplePointP
params.esat = lambda T: pyrads.Thermodynamics.get_satvps(T,params.satvap_T0,params.satvap_e0,params.Rv,params.Lvap)

params.R = pyrads.phys.air.R  # dry component
params.cp = pyrads.phys.air.cp
params.ps_dry = 1e5           # surface pressure of dry component

params.g = 9.8             # surface gravity
params.cosThetaBar = 3./5. # average zenith angle used in 2stream eqns
params.RH = 1.             # relative humidity

# setup resolution
N_press = 15
dwavenr = 0.1

wavenr_min = 0.1   # [cm^-1]
wavenr_max = 3500. #

Tstrat = 150.      # stratospheric temperature
Ts = 300.

# setup grid:
g = pyrads.SetupGrids.make_grid( Ts,Tstrat,N_press,wavenr_min,wavenr_max,dwavenr,params, RH=params.RH )

In [27]:
# compute optical thickness:
#   -> this is the computationally most intensive step
g.tau = pyrads.OpticalThickness.compute_tau_H2ON2(g.p,g.T,g.q,g,params, RH=params.RH )

# compute Planck functions etc:
T_2D = np.transpose(np.tile( g.T, (g.Nn,1) )) # shape=(g.p,g.n)
g.B_surf = np.pi* pyrads.Planck.Planck_n( g.n,Ts ) # shape=(g.n)
g.B = np.pi* pyrads.Planck.Planck_n( g.wave, T_2D ) # shape=(g.p,g.n)

# compute OLR etc:
olr_spec = pyrads.Get_Fluxes.Fplus_alternative(0,g) # (spectrally resolved=irradiance)
olr = simps(olr_spec,g.n)

print( "OLR = ",olr)


compute kappa at p,T =  10.358134708328855 150.0
compute kappa at p,T =  19.99842715683728 150.0
compute kappa at p,T =  38.610917892942844 150.0
compute kappa at p,T =  74.54601148600246 150.0
compute kappa at p,T =  143.92581507332955 150.0
compute kappa at p,T =  277.877244289748 150.0
compute kappa at p,T =  536.4969644585528 150.0
compute kappa at p,T =  1035.8134708328855 150.0
compute kappa at p,T =  1999.842715683728 150.0
compute kappa at p,T =  3861.0917892942844 150.0
compute kappa at p,T =  7454.601148600237 172.28628151273767
compute kappa at p,T =  14392.581507332956 207.7982782533632
compute kappa at p,T =  27787.724428974798 246.63230604746863
compute kappa at p,T =  53649.69644585523 276.90876054505844
compute kappa at p,T =  103581.34708328855 299.99999999999994
done! 

OLR =  279.5793345129229
